## Tokenization Preparation
Data cleaning for lyrics. This is to apply language detection and '[]' bracket removal to a dataframe which already has lyrics, and further apply tokenization and sentiment scoring.

Works well, need to convert to .py script

In [2]:
# Cleaning:
# Not all songs same length
# Not all have [Verse 1] or [Intro] as start
# -if using '[' as strip point
# -if not, can use 'Lyrics' as strip point
# if neither, leave it, and list locations (to check results)
# If they have a '[' in the lyrics, strip up to the first one
# words with dashes '-' or in parentheses '()' or #
# words in other languages

# Custom STOP words
# [Intro, Chorus, Verse #, Bridge, Interlude, x#, Pre-Chorus]

In [3]:
import pandas as pd
import langdetect

df = pd.read_csv("output.csv")
df.drop(columns='lyrics_found')
df.head()

C:\Users\Owner\AppData\Local\Temp\ipykernel_27108\513083978.py:4: DtypeWarning: Columns (7,8,9,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("output.csv")


,title,duration,album,release_date,year,artist,id,country,genre,style,source,lyrics,lyrics_found
0,Last Dance Of A Dying King,NaN,Anthems Of Resistance,NaN,2007.0,Wolfpack Unleashed,4274581,Europe,Rock,Thrash,discogs,NaN,NaN
1,Religion Of Control (Radio Edit),NaN,Anthems Of Resistance,NaN,2007.0,Wolfpack Unleashed,4274581,Europe,Rock,Thrash,discogs,NaN,NaN
2,Next Victim,NaN,Anthems Of Resistance,NaN,2007.0,Wolfpack Unleashed,4274581,Europe,Rock,Thrash,discogs,NaN,NaN
3,Killing Fields,NaN,Anthems Of Resistance,NaN,2007.0,Wolfpack Unleashed,4274581,Europe,Rock,Thrash,discogs,NaN,NaN
4,Disgrace Erased,NaN,Anthems Of Resistance,NaN,2007.0,Wolfpack Unleashed,4274581,Europe,Rock,Thrash,discogs,NaN,NaN


In [4]:
from langdetect import detect
def safe_detect(text):
    try:
        return detect(text)
    except:
        return "unknown"

df['language'] = df['lyrics'].apply(safe_detect)

In [5]:
df_subset = df[df['lyrics'].notna()].copy()
df_subset.shape

(9685, 14)

In [6]:
# may need to alter this choice, as there are multi-lingual tracks ending up in here
df_subset = df_subset[df_subset['language'] == 'en'].reindex()
df_subset.shape

(6731, 14)

In [7]:
# Pre-check of troublemakers
print(df_subset['lyrics'].iloc[1], '\n-----')
print(df_subset['lyrics'].iloc[30], '\n-----')
print(df_subset['lyrics'].iloc[40], '\n-----')
print(df_subset['lyrics'].iloc[50], '\n-----')
print(df_subset['lyrics'].iloc[80])

2 Contributors2 Nigs United 4 West Compton (One Nite Alone Tour... The Aftershow) Lyrics[Intro]
Two, three, get on it now

[Instrumental]

[Ad lib]
I wanna sing, but it's too funky!

[Spoken]
Two nigs united for West Compton

[Instrumental]

[Ad lib]
C'mon, y'all!

[Instrumental]

[Spoken]
On the one!
One, two, three, get up
On the one!
Keep that going! 
-----
1 ContributorBlue Hotel LyricsSome paradise that draws me on
As far as I can go
Some act of god gone crazy now
For all the love in the world

Like a traveler I've come
I am just a child waitin' on the bus
Find the two of us inside the blue hotel
Where I paid my respects
And I never went to sleep all night

A lost wilderness that haunts me
Where I can't find my breath
Some sacrifice is needed before I can go on
Down the corridors I walk
I'm searching for the right number on the door

It's just another night inside the blue hotel
Where I paid my respects
And we both stayed out of sight
And I never went to sleep all night

Some para

In [8]:
import re

unmatched_lyrics = []

def clean_lyrics(text):
    if pd.isna(text):
        return text
    match = re.search(r'\[.*?\]', text)
    match1 = re.search(r'Lyrics', text)
    if match:
        return text[match.start():].strip()
    elif match1:
        return text[match1.end():].strip()
    else:
        unmatched_lyrics.append(text)
        return text.strip()  # Return original text if no bracketed section

df_subset['clean_lyrics'] = df_subset['lyrics'].apply(clean_lyrics)

In [9]:
# Re-check to confirm the troublemakers were cleaned
print("Number of tracks that don't meet either condition:", len(unmatched_lyrics))
print(df_subset['clean_lyrics'].iloc[1], '\n-----')
print(df_subset['clean_lyrics'].iloc[30], '\n-----')
print(df_subset['clean_lyrics'].iloc[40], '\n-----')
print(df_subset['clean_lyrics'].iloc[50], '\n-----')
print(df_subset['clean_lyrics'].iloc[80])

Number of tracks that don't meet either condition: 0
[Intro]
Two, three, get on it now

[Instrumental]

[Ad lib]
I wanna sing, but it's too funky!

[Spoken]
Two nigs united for West Compton

[Instrumental]

[Ad lib]
C'mon, y'all!

[Instrumental]

[Spoken]
On the one!
One, two, three, get up
On the one!
Keep that going! 
-----
Some paradise that draws me on
As far as I can go
Some act of god gone crazy now
For all the love in the world

Like a traveler I've come
I am just a child waitin' on the bus
Find the two of us inside the blue hotel
Where I paid my respects
And I never went to sleep all night

A lost wilderness that haunts me
Where I can't find my breath
Some sacrifice is needed before I can go on
Down the corridors I walk
I'm searching for the right number on the door

It's just another night inside the blue hotel
Where I paid my respects
And we both stayed out of sight
And I never went to sleep all night

Some paradise that holds me
I sealed it with a kiss
Some sacrifice is need

In [10]:
# Need to create a list of all bracketed words for unique stopword list
unique_brackets = set().union(*df_subset['clean_lyrics'].dropna().apply(
    lambda x: set(re.findall(r'\[.*?\]', x))
))

print(f"Found {len(unique_brackets)} unique bracketed phrases:")
print(sorted(unique_brackets))

Found 3356 unique bracketed phrases:
['[ Bridge ]', '[ Hook 2x ]', '[ Hook ]', '[ J. J. Johnson ]', '[ Verse 1 ]', '[ Verse 2 ]', '[ Verse 3 ]', '[ verse 3 frank ocean]', '["danny boy..."]', '[*Gunshot*]', "[*I'm Sprung Ringtone*]", '[...]', '[10-FEET「SHOES」歌詞]', '[12" Mix]', '[12" Version]', '[1975]', '[1st Chorus]', '[2014 Remaster part starting]', '[2Pac talking]', '[2nd Chorus]', '[2x]', '[3rd Chorus]', '[3x]', '[4th Chorus]', '[4x]', '[50 Cent talking]', '[5x]', '[98°’s Version]', '[? ??? chromosomes ??? ?]', '[? Alice topless cries ?]', '[? Antique/Antic Christmas eyes ?]', '[? Yes/Guess ?]', '[? ]', '[? are ?]', '[? fate was ??? ?]', "[? karma's ?]", '[? liar(s)/lyre(s) ?]', '[? some ??? ?]', '[? your mind/line ?]', '[????? incomprehensible screaming]', '[???]', '[??]', '[?]', '[?x]', '[A Man Needs a Maid]', '[ALL STUDENTS]', '[AMY & TERRY]', '[AMY, spoken]', '[AMY]', '[Accordion and Pedal Steel Guitar Solos]', '[Ad lib]', '[Al Barr (Murpheys)]', '[Album Intro: Danny from "Brass

In [11]:
# oookay, looks like it will be better to remove them instead to have clean lyrics to work with.
# Now we need to figure out how to discard repetitions in lyrics, since they don't add anything to 
# change the sentiment or understanding.

df_subset['clean_lyrics'] = (
    df_subset['clean_lyrics']
    .str.replace(r'\[.*?\]', '', regex=True)         # Remove all bracketed phrases
    .str.replace(r'\s+', ' ', regex=True)            # Normalize extra spaces
    .str.strip()                                     # Trim leading/trailing spaces
)

In [12]:
# Double-check that it worked
check_brackets = set().union(*df_subset['clean_lyrics'].dropna().apply(
    lambda x: set(re.findall(r'\[.*?\]', x))
))

print(f"Found {len(check_brackets)} bracketed phrases:")
print(sorted(check_brackets))

Found 4 bracketed phrases:
["[Black burnt flesh Hark hark Crack unit species Who makes the Nazis? (All the O's cross country) Who makes the Nazis? Shag-artists]", '[Spin slack through reality) (Dance my way, falling up through the years]', "[Till I swivel back ‘round, then I fly, fly, fly) Now I'm wiser than dreams (Losing breath underwater, well, I'm gone, gone, gone) Let me fly, fly, fly (Spin slack through reality) While I'm touching tomorrow (Dance my way, falling up through the years) And I know who's there [Till I swivel back ‘round, then I fly, fly, fly) (Losing breath underwater when I'm gone, gone, gone) When silhouettes fall (Spinning slack through reality) And I'm gone Like I'm dancing on angels And I'm gone through the crack in the past Like a dead man walking Like a dead man walking, oh ho Three old men dancing under the lamp lights Shaking their sex and the bones And the boys that we were An alien nation in therapy Sliding naked anew Like a bad-tempered child On the rain-

## Tokenization and sentiment scoring

In [20]:
import gensim
from transformers import pipeline
from tqdm import tqdm


In [22]:
# Transformers (SST-2)
transformer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
def transformer_sentiment(text):
    if pd.isna(text):
        return None
    result = transformer(text[:512])[0]  # Truncate to 512 tokens max
    return result['score'] if result['label'] == 'POSITIVE' else -result['score']

In [24]:
df_subset['sentiment_score'] = df_subset['clean_lyrics'].apply(lambda lyric: transformer_sentiment(lyric.strip()) if pd.notna(lyric) else None)

In [26]:
df_subset.head()

,title,duration,album,release_date,year,artist,id,country,genre,style,source,lyrics,lyrics_found,language,clean_lyrics,sentiment_score
13,El Tiempo,NaN,Fuerza Para Seguir,NaN,2007.0,Unidad 4,3073827,Peru,Rock,"Hardcore, Punk",discogs,3 ContributorsIris LyricsSo the song title and...,NaN,en,Ayo! Korgy! Shir! You know this 1337M0B Music ...,0.756323
15,Poder,NaN,Fuerza Para Seguir,NaN,2007.0,Unidad 4,3073827,Peru,Rock,"Hardcore, Punk",discogs,2 Contributors2 Nigs United 4 West Compton (On...,NaN,en,"Two, three, get on it now I wanna sing, but it...",-0.986049
33,Fuckin' Aye Reloaded 4.0,178.0,Story Of Ohm,NaN,2007.0,Paniq,1151278,Germany,Electronic,"Breakbeat, Psy-Trance, Techno, Chiptune, Exper...",discogs,1 ContributorFuckin’ Aye Reloaded 4.0 LyricsYo...,NaN,en,You can't walk away now Hahaha...,0.998131
76,Flieg,NaN,10 Jaar Live / Simply Me,NaN,2007.0,Nádine,31648628,South Africa,Pop,NaN,discogs,1 ContributorLord Of The Flies Lyrics[Verse 1]...,NaN,en,I’ve led a crocodile smile life Grin to the fa...,-0.924693
78,You And I,NaN,10 Jaar Live / Simply Me,NaN,2007.0,Nádine,31648628,South Africa,Pop,NaN,discogs,1 ContributorYou and I Lyrics[Prehook: Jez]\n\...,NaN,en,You and I say A little fun and some Bombay Tha...,0.946630


In [28]:
#df_subset.to_csv("output_clean_lyrics.csv")